<a href="https://colab.research.google.com/github/Guillaume-nt/pywritings/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yolov5 Object Detection Model

## Installation

### Virtual environment

In [1]:
#Create a virtual environment
!pip install virtualenv

     |████████████████████████████████| 5.6 MB 5.2 MB/s 
     |████████████████████████████████| 461 kB 59.8 MB/s 


In [2]:
#Name the environment decicaded to yolov5 use
!virtualenv yolov5_env

created virtual environment CPython3.7.12.final.0-64 in 856ms
  creator CPython3Posix(dest=/content/yolov5_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==21.3.1, setuptools==58.3.0, wheel==0.37.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [3]:
#Install the required packages in the environement
#Need to change environment variables
!yolov5_env\Scripts\activate

/bin/bash: yolov5_envScriptsactivate: command not found


### Cloning repository

In [4]:
#Clone the repository
#The repository is clone to use a yolo model which is configured
!git clone https://github.com/Guillaume-nt/pywritings

Cloning into 'pywritings'...
remote: Enumerating objects: 4048, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 4048 (delta 81), reused 2 (delta 0), pack-reused 3851
Receiving objects: 100% (4048/4048), 1.01 GiB | 37.71 MiB/s, done.
Resolving deltas: 100% (435/435), done.
Checking out files: 100% (3370/3370), done.


### Dependencies installation 

In [5]:
#Install requirements from de yolov5 repository (list of all required libraries to train the model)
!pip install -r /content/pywritings/yolov5/requirements.txt

     |████████████████████████████████| 596 kB 5.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
!pip install torch==1.10.0+cpu torchvision==0.11.1+cpu torchaudio==0.10.0+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cpu/torch_stable.html
     |████████████████████████████████| 199.3 MB 64 kB/s 
     |████████████████████████████████| 16.2 MB 30.2 MB/s 
     |████████████████████████████████| 2.7 MB 1.2 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0+cu111
    Uninstalling torchaudio-0.10.0+cu111:
      Successfully uninstalled torchaudio-0.10.0+cu111


## Convert CSV annotations to YOLOformat

In [7]:
#Import

import os
import numpy as np 
import pandas as pd
import shutil as sh
from PIL import Image
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from google.colab import drive

In [8]:
df = pd.read_csv('/content/pywritings/forms.csv')
df_box = pd.read_csv('/content/pywritings/word.csv')

In [9]:
df.head()

,form id,writer id,number of sentences,word segmentation,NBTO,NBOK,WOTO,WOOK,path
0,a01-000u,0,2,prt,7,5,52,36,forms/a01-000u.png
1,a01-000x,1,2,all,6,6,52,52,forms/a01-000x.png
2,a01-003,2,3,prt,11,10,81,71,forms/a01-003.png
3,a01-003u,0,3,prt,11,10,80,73,forms/a01-003u.png
4,a01-003x,3,3,prt,9,6,82,56,forms/a01-003x.png


In [10]:
df_box.head()

,fileName,out_of_focus,greyLevel,x_min,y_min,width,height,tag,word,len,space_in_text,path
0,a01-000u-00-00,1,154,408,768,27,51,AT,A,1,0,words/a01/a01-000u/a01-000u-00-00.png
1,a01-000u-00-01,1,154,507,766,213,48,NN,MOVE,4,0,words/a01/a01-000u/a01-000u-00-01.png
2,a01-000u-00-02,1,154,796,764,70,50,TO,to,2,0,words/a01/a01-000u/a01-000u-00-02.png
3,a01-000u-00-03,1,154,919,757,166,78,VB,stop,4,0,words/a01/a01-000u/a01-000u-00-03.png
4,a01-000u-00-04,1,154,1185,754,126,61,NPT,Mr.,3,0,words/a01/a01-000u/a01-000u-00-04.png


In [11]:
#Data preparation to train the yolov5 model with our labels

df_box = df_box[df_box["out_of_focus"]==1]
df_box = df_box[['fileName','path','x_min','y_min','width','height', 'word']]

df_box['path']=df_box['path'].apply(lambda x : x.rsplit("/")[2])

df_box['x_center'] = (df_box['x_min']+df_box['width'])/2/2479
df_box['y_center'] = (df_box['y_min']+df_box['height'])/2/3542
df_box['width']=df_box['width']/2479
df_box['height']=df_box['height']/3542

df_f = df_box[['path','x_center','y_center','width','height', 'word']]
df_f

,path,x_center,y_center,width,height,word
0,a01-000u,0.087737,0.115613,0.010891,0.014399,A
1,a01-000u,0.145220,0.114907,0.085922,0.013552,MOVE
2,a01-000u,0.174667,0.114907,0.028237,0.014116,to
3,a01-000u,0.218838,0.117871,0.066962,0.022021,stop
4,a01-000u,0.264421,0.115048,0.050827,0.017222,Mr.
...,...,...,...,...,...,...
115315,r06-143,0.279750,0.212451,0.025817,0.021739,him
115316,r06-143,0.301331,0.214709,0.014522,0.014681,in
115317,r06-143,0.323719,0.214145,0.029447,0.020327,the
115318,r06-143,0.373739,0.219932,0.080274,0.032185,garden


In [12]:
df_label = df_f
df_label['classes']=0
df_label = df_f[['path','classes','x_center','y_center','width','height']]
df_label.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,path,classes,x_center,y_center,width,height
0,a01-000u,0,0.087737,0.115613,0.010891,0.014399
1,a01-000u,0,0.145220,0.114907,0.085922,0.013552
2,a01-000u,0,0.174667,0.114907,0.028237,0.014116
3,a01-000u,0,0.218838,0.117871,0.066962,0.022021
4,a01-000u,0,0.264421,0.115048,0.050827,0.017222


## YAML file creation for training

In [14]:
#Creation of COCO file
#Conversion of images in jpg and done in the repository

df = df_f[['path','x_center','y_center','width','height','word']]

## creating a column name image_id having images names as id 
df['image_id'] = df['path']
df['classes'] = 'word'
labels = df['classes'].unique()
labels_to_dict = dict(zip(labels, range(0, len(labels))))
print('Lables Directory:', labels_to_dict)

df.replace({'classes':labels_to_dict}, inplace=True)
df = df[['image_id','x_center','y_center','width','height','classes']]
## set index of images
index = list(set(df.image_id))

Lables Directory: {'word': 0}


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
#done in repo
os.chdir("/content/pywritings/yolov5/txt")
data_path = "/content/drive/MyDrive/Git/Pywritings/yolov5/forms_jpg"


source = 'train'
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(df.groupby('image_id')):
            if name in val_index:
                path2save = 'val1/'
            else:
                path2save = 'train1/'
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','width','height']].astype(float).values
                imagename = data_path+"/{}.jpg".format(name)
                check_image_width_height = Image.open(imagename)
                img_width, img_height = check_image_width_height.size
                #for r in (row):
                    #r[1] = r[1]/img_width
                    #r[2] = r[2]/img_height
                    #r[3] = r[3]/img_width
                    #r[4] = r[4]/img_height
                row = row.astype(str)
                for j in range(len(row)):
                    print(row[j], '/n')
                    row[j][0] = str(int(float(row[j][0])))
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write('\n')
                   # f.write("n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
            sh.copy(data_path+"/{}.jpg".format(name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))

  0%|          | 0/1539 [00:00<?, ?it/s]

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
['0.0' '0.1399757966922146' '0.1647374364765669' '0.0572811617587737'
 '0.021456804065499716'] /n
['0.0' '0.16841468334005646' '0.16459627329192547' '0.026623638563937072'
 '0.017504234895539244'] /n
['0.0' '0.2127874142799516' '0.16826651609260304' '0.07462686567164178'
 '0.026538678712591756'] /n
['0.0' '0.2444534086325131' '0.1671372106154715' '0.033481242436466316'
 '0.026256352343308864'] /n
['0.0' '0.29689390883420735' '0.17546583850931677' '0.09156918112141993'
 '0.042631281761716544'] /n
['0.0' '0.32573618394513915' '0.16614906832298137' '0.03227107704719645'
 '0.021456804065499716'] /n
['0.0' '0.3590157321500605' '0.16445511010728403' '0.04759983864461476'
 '0.020045172219085264'] /n
['0.0' '0.4302137958854377' '0.1717955957086392' '0.11859620814844696'
 '0.03274985883681536'] /n
['0.0' '0.08350141185962082' '0.18929983060417843' '0.02743041549011698'
 '0.019762845849802372'] /n
['0.0' '0.12787414279

In [ ]:
#done in repo

os.chdir("/content/yolov5/data")

with open("coco.txt", "r") as source, open("coco.yml", "w") as dest:
    dest.write(source.read())

In [ ]:
#Install Weight & Bias
#Need to get a Weight & Bias API key through that link : https://wandb.ai/site
!pip install wandb
!wandb login 90d45a6ecee90282b0e52346c1e7014dcf0f7c58

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
os.chdir("/content/drive/MyDrive/Git/Pywritings")
os.getcwd()

'/content/drive/MyDrive/Git/Pywritings'

In [ ]:
#Train the model en choose de parameters
os.chdir("/content/drive/MyDrive/Git/Pywritings/yolov5")

!python /content/drive/MyDrive/Git/Pywritings/yolov5/train.py --img 416 --batch 12 --epochs 1 --data coco.yml --weights ./weights/yolov5x.pt
#!python yolov5/train.py --img 416 --batch 12 --epochs 50 --data yolov5/data/coco.yml --weights ./weights/yolov5x.pt

wandb: Currently logged in as: guillaume_nt (use `wandb login --relogin` to force relogin)
train: weights=./weights/yolov5x.pt, cfg=, data=coco.yml, hyp=data/hyps/hyp.scratch.yaml, epochs=20, batch_size=12, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 2021-11-29 torch 1.7.0+cpu CPU

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor

In [ ]:
#Train the model en choose de parameters

!python /content/pywritings/yolov5/train.py --img 416 --batch 12 --epochs 10 --data /content/pywritings/coco.yml --weights ./weights/yolov5s.pt

train: weights=./weights/yolov5s.pt, cfg=, data=/content/pywritings/coco.yml, hyp=pywritings/yolov5/data/hyps/hyp.scratch.yaml, epochs=10, batch_size=12, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=pywritings/yolov5/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 c51bdc8 torch 1.7.0+cpu CPU

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv

In [ ]:
#Train the model en choose de parameters

!python /content/pywritings/yolov5/train.py --img 640 --batch 12 --epochs 10 --data /content/pywritings/coco.yml --weights ./weights/yolov5s.pt

wandb: Currently logged in as: guillaume_nt (use `wandb login --relogin` to force relogin)
train: weights=./weights/yolov5s.pt, cfg=, data=/content/pywritings/coco.yml, hyp=pywritings/yolov5/data/hyps/hyp.scratch.yaml, epochs=10, batch_size=12, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=pywritings/yolov5/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 c51bdc8 torch 1.10.0+cpu CPU

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0

In [ ]:
#Train the model en choose de parameters

!python /content/pywritings/yolov5/train.py --img 416 --batch 12 --epochs 10 --data /content/pywritings/coco.yml --weights ./weights/yolov5s.pt

In [ ]:
import torch

In [ ]:
 #Test the model

!python /content/pywritings/yolov5/detect.py --img 416 --source /content/pywritings/yolov5/detect/test.jpg --weights /content/pywritings/yolov5/weights/last.pt --conf-thres 0.6
 

detect: weights=['/content/pywritings/yolov5/weights/last.pt'], source=/content/pywritings/yolov5/detect/test.jpg, imgsz=[416, 416], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=pywritings/yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 c51bdc8 torch 1.10.0+cpu CPU

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/pywritings/yolov5/detect/test.jpg: 416x320 1 word, Done. (0.144s)
Speed: 0.9ms pre-process, 144.0ms inference, 0.8ms NMS per image at shape (1, 3, 416, 416)
Results saved to pywritings/yolov5/runs/detect/exp16


In [ ]:
!pip install pillow  
!pip install pytesseract